In [2]:
from suds.client import Client
from suds.transport.http import HttpTransport
import urllib2

In [3]:
class HTTPSudsPreprocessor(urllib2.BaseHandler):
    def __init__(self, SID):
        self.SID = SID

    def http_request(self, req):
        req.add_header('cookie', 'SID="'+self.SID+'"')
        return req

    https_request = http_request

In [4]:
class WokmwsSoapClient():
    """
    main steps you have to do:
        soap = WokmwsSoapClient()
        results = soap.search(...)
    """
    def __init__(self):
        self.url = self.client = {}
        self.SID = ''

        #self.url['auth'] = 'http://search.isiknowledge.com/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
        self.url['auth'] = 'http://search.webofknowledge.com/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
        #self.url['search'] = 'http://search.isiknowledge.com/esti/wokmws/ws/WokSearchLite?wsdl'
        self.url['search'] = 'http://search.webofknowledge.com/esti/wokmws/ws/WokSearchLite?wsdl'

        self.prepare()

    def __del__(self):
        self.close()

    def prepare(self):
        """does all the initialization we need for a request"""
        self.initAuthClient()
        self.authenticate()
        self.initSearchClient()

    def initAuthClient(self):
        self.client['auth'] = Client(self.url['auth'])

    def initSearchClient(self):
        http = HttpTransport()
        opener = urllib2.build_opener(HTTPSudsPreprocessor(self.SID))
        http.urlopener = opener
        self.client['search'] = Client(self.url['search'], transport = http)

    def authenticate(self):
        self.SID = self.client['auth'].service.authenticate()

    def close(self):
        self.client['auth'].service.closeSession()

    def search(self, query):
        qparams = {
            'databaseId' : 'WOS',
            'userQuery' : query,
            'queryLanguage' : 'en',
            'editions' : [{
                'collection' : 'WOS',
                'edition' : 'SCI',
            },{
                'collection' : 'WOS',
                'edition' : 'SSCI',
            }]
        }

        rparams = {
            'count' : 5, # 1-100
            'firstRecord' : 1,
            #'fields' : [{
            #    'name' : 'Relevance',
            #    'sort' : 'D',
            #}],
        }

        return self.client['search'].service.search(qparams, rparams)

In [5]:
soap = WokmwsSoapClient()

In [6]:
results = soap.search('AU=Hallam')

In [42]:
dir(results)

['__contains__',
 '__delattr__',
 '__doc__',
 '__getitem__',
 '__init__',
 '__iter__',
 '__keylist__',
 '__len__',
 '__metadata__',
 '__module__',
 '__printer__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__str__',
 '__unicode__',
 'queryId',
 'records',
 'recordsFound',
 'recordsSearched']

In [9]:
print results.recordsFound

1961